In [1]:
import os
import cv2
import numpy as np
import random
import sys
import testGMM
import re
import copy

In [2]:
# parameters:
# k: int, number of guassian distribution
# max_iter: int, maximum number of step in optimization
# img_name: strings, the relative path to single image, i.e. "train_images/032.jpg"
def trainGMM(K, max_iter, img_name):
    # read img
    img = cv2.imread(img_name)
    # user defined converge threshold
    tau = 0.00000000000000001

    def initialize():
        mean = np.asmatrix([[random.randint(1, 255)],[random.randint(1, 255)],[random.randint(1, 255)]])
        # generate a random positive-semidefinete matrix as covariance matrix
        A = np.random.random((3,3))*20
        cov = np.dot(A, A.transpose())
        scaling = random.random() * 5.0
        return [scaling,mean,cov]

    params = [initialize() for cluster in range(K)]
    # Structure of para:
    # [[scale,mean,covariance],[scale,mean,covariance],[scale,mean,covariance]...]
    # scale is a int. Mean is a 3x1 matrix. Covariance is a 3x3 matrix

    # total_mean is the sum of all mean from different clusters
    total_mean = np.full((K,3, 3), -9999)
    prev_total_mean = np.full((K,3, 3), 9999)
    iter = 0

    # return true if MLE converges. Return false otherwise
    def check_convergence(total_mean, prev_toal_mean, tau):
        sum = 0
        print("For cluster 1, total_mean and Prev_total_mean are as follow\n")
        print("curr mean: ", total_mean[0])
        print("prev mean: ", prev_total_mean[0])
        for cluster in range(len(prev_toal_mean)):
            sum += np.linalg.norm(total_mean[cluster]-prev_total_mean[cluster])
        print("Check convergence difference: ", sum)
        return sum >= tau

    while iter < max_iter and check_convergence(total_mean,prev_total_mean,tau):
        print("iter: ", iter)
        # update prev total mean
        prev_total_mean = copy.deepcopy(total_mean)

        # Expectation step - assign points to clusters, get cluster weight
        weights = [ [] for _ in range(K) ]
        for cluster in range(K):
            # weight for a single cluster
            cluster_weights = np.zeros((img.shape[0], img.shape[1]))
            # cumulated weights add up all weights on a given pixel -- serving as denominator
            cumulated_weights = np.zeros((img.shape[0], img.shape[1]))
            cluster_scaling, cluster_mean, cluster_cov = params[cluster]
            # print("Current scaliing for cluster ", cluster_scaling)
            # print("Current mean for cluster ", cluster_mean)
            # print("Current cov for cluster \n", cluster_cov)


            for w in range(len(img[:, 0, 0])):
                for h in range(len(img[0, :, 0])):
                    pix = np.asmatrix([[img[w][h][0]], [img[w][h][1]], [img[w][h][2]]])
                    try:
                        likelihood = testGMM.get_likelihood(pix, cluster_mean, cluster_cov)
                    except:
                        likelihood = 0
                    likelihood = likelihood if likelihood != 0 else sys.float_info.min
                    # if likelihood == 0:
                    #     print("likelihood is zero")
                    #     print("curr_pix: \n",pix)
                    #     print("cluster_mean: \n", cluster_mean)
                    #     print("cluster cov: \n", cluster_cov)
                    #     sys.exit()
                    ## calculate weight at position (w, h)
                    weight = cluster_scaling * likelihood
                    cumulated_weights[w][h] += weight
                    cluster_weights[w][h] = weight # probability of each pixel belonging to this cluster

            weights[cluster] = cluster_weights # weights for all clusters 1 to K,
            #weights[i][w][h]is the probability of the (w,h) pixel belonging to the ith cluster
        for cluster in range(K):
            weights[cluster] = np.divide(weights[cluster], cumulated_weights)

        # Maximization step - get new scaling, mean, and cov for each cluster
        for cluster in range(K):
            mean_sum = np.zeros((3,1)) # sums all weight*pixel RGB value on image
            cov_sum = np.zeros((3,3))
            sum_weights = np.sum(weights[cluster]) # sum of all the weights given a cluster
            for w in range(len(img[:, 0, 0])):
                for h in range(len(img[0, :, 0])):
                    pix = np.asmatrix([[img[w][h][0]], [img[w][h][1]], [img[w][h][2]]])
                    # calculate mean
                    mean_sum += np.multiply(weights[cluster][w][h],pix)

            new_mean = np.divide(mean_sum, sum_weights)

            for w in range(len(img[:, 0, 0])):
                for h in range(len(img[0, :, 0])):
                    pix = np.asmatrix([[img[w][h][0]], [img[w][h][1]], [img[w][h][2]]])
                    # calculate covariance
                    cov_sum += np.multiply(weights[cluster][w][h], (pix - new_mean))@((pix - new_mean).T)
            new_cov = np.divide(cov_sum, sum_weights)
            new_scaling = sum_weights / (img.shape[0]*img.shape[1])
            mean_sum += mean_sum

            total_mean[cluster] = new_mean
            print("new mean at cluster ", cluster, "is ", new_mean )
            # update model
            params[cluster] = (new_scaling, new_mean, new_cov)
        print("-----------------")
        iter += 1
    # store weights to .npy
    if not os.path.exists("weights"):
        os.mkdir("weights")
    else:
        digit = re.findall(r'\d+\d+\d*',img_name)
        file_name = str(digit[0])+"_weight.npy"
        with open(os.path.join("weights",file_name), "wb") as f:
            np.save(f,params)

In [3]:
input_dir = "train_images"
for img_name in os.listdir(input_dir):
    img = os.path.join(input_dir, img_name)
    trainGMM(2,200,img)
    print("Finish Training for ", img)
    break

For cluster 1, total_mean and Prev_total_mean are as follow

curr mean:  [[-9999 -9999 -9999]
 [-9999 -9999 -9999]
 [-9999 -9999 -9999]]
prev mean:  [[9999 9999 9999]
 [9999 9999 9999]
 [9999 9999 9999]]
Check convergence difference:  119988.0
iter:  0
pixel
  [[0]
 [0]
 [0]]
mean
  [[ 35]
 [120]
 [ 16]]
cov
  [[177.79559689 274.45152551 324.01227339]
 [274.45152551 453.37541058 505.25593139]
 [324.01227339 505.25593139 714.20696346]]
pixel
  [[0]
 [0]
 [0]]
mean
  [[57]
 [92]
 [93]]
cov
  [[906.80394172 457.68186477 767.37745175]
 [457.68186477 263.46068557 393.13375512]
 [767.37745175 393.13375512 759.99667535]]
new mean at cluster  0 is  [[0.00013743]
 [0.00025246]
 [0.00040267]]
new mean at cluster  1 is  [[132.05619466]
 [133.52069987]
 [132.32685547]]
-----------------
For cluster 1, total_mean and Prev_total_mean are as follow

curr mean:  [[0 0 0]
 [0 0 0]
 [0 0 0]]
prev mean:  [[-9999 -9999 -9999]
 [-9999 -9999 -9999]
 [-9999 -9999 -9999]]
Check convergence difference:  60391.

/home/da/cmsc426/hw1env/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


with open('weights/68_weight.npy', 'rb') as f:
    a = np.load(f, allow_pickle=True)
    print(a[2])

In [4]:
import testGMM
import math
pix = np.asmatrix([[157],[140],[121]])
mean =  np.asmatrix([[6], [191], [140]])
cov =  np.asmatrix([[1154 ,1497 ,1072], [1497, 2330 ,1198], [1072 ,1198, 1123]])
likelihood = testGMM.get_likelihood(pix, mean, cov)
print("likelihood: ", likelihood)
print((-0.5) * (pix - mean).T @ np.linalg.inv(cov) @ (pix - mean))

likelihood:  0.0
[[-1346.9566598]]


In [5]:
import numpy as np
A = [ [] for _ in range(3) ]
A[1] = [[3],[2],[2]]
print(A)

[[], [[3], [2], [2]], []]
